In [2]:
pip install wfdb

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/160.0 kB ? eta -:--:--
   -- ------------------------------------- 10.2/160.0 kB ? eta -:--:--
   --------- ----------------------------- 41.0/160.0 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- - 153.6/160.0 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 160.0/160.0 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.0 MB 4.5 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.0 MB 3.6 MB/s eta 0:00:01
   ---------------- ----------------------- 0.4/1.0 MB 3.2 MB/s eta 0:00:01
   --------------------------- ------------ 0.7/1.0 MB 3.9 MB/s eta 0:00:01
   ------------------------------------- -- 1.0/1.0 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.0 MB/s eta 0:00:00
Note: you may 

In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/197.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/197.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/197.9 MB 3.0 MB/s eta 0:01:07
   ---------------------------------------- 0.5/197.9 MB 4.4 MB/s eta 0:00:45
   ---------------------------------------- 0.8/197.9 MB 5.2 MB/s eta 0:00:38
   ---------------------------------------- 1.2/197.9 MB 6.2 MB/s eta 0:00:32
   ---------------------------------------- 1.5/197.9 MB 6.5 MB/s eta 0:00:31
   ---------------------------------------- 1.9/197.9 MB 6.6 MB/s eta 0:00:30
   ---------------------------------------- 2.5/197.9 MB 7.9 MB/s eta 0:00:25
    --------------------------------------- 3.1/197.9 MB 8.6 MB/s eta 0:00:23
    --------------------------------------- 3.7/197.9 MB 9.1 MB/s eta 0:00:22
    --------------------------------------- 4.8/197.9 MB 10.2 MB/s eta 0:00:19

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
pip uninstall torch

In [1]:
import numpy as np
import wfdb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
import pandas as pd

# Define the ECGDataset class
class ECGDataset(Dataset):
    def __init__(self, raw_signals, noisy_signals):
        self.raw_signals = raw_signals
        self.noisy_signals = noisy_signals

    def __len__(self):
        return len(self.raw_signals)

    def __getitem__(self, idx):
        raw_signal = self.raw_signals[idx]
        noisy_signal = self.noisy_signals[idx]
        return torch.tensor(raw_signal, dtype=torch.float32), torch.tensor(noisy_signal, dtype=torch.float32)

# Define the add_noise function
def add_noise(signal, noise, snr):
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    factor = (signal_power / noise_power) / (10 ** (snr / 10))
    noisy_signal = signal + noise * np.sqrt(factor)
    return noisy_signal

# Define the load_mit_bih_data function
def load_mit_bih_data(records, noise_type, snr_levels, target_length=650000):
    raw_signals = []
    noisy_signals_dict = {snr: [] for snr in snr_levels}
    
    for record in records:
        raw_record = wfdb.rdrecord(f'M:\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{record}')
        raw_signal = raw_record.p_signal[:, 0]  # Use the first channel for simplicity
        e
        # Load noise and add it to the raw signal
        noise_record = wfdb.rdrecord(f'M:\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0/{noise_type}')
        noise_signal = noise_record.p_signal[:, 0]
        
        # Ensure the signals are of the same length
        min_length = min(len(raw_signal), len(noise_signal), target_length)
        raw_signal = raw_signal[:min_length]
        noise_signal = noise_signal[:min_length]
        
        # Pad signals to target length
        if min_length < target_length:
            raw_signal = np.pad(raw_signal, (0, target_length - min_length), 'constant')
            noise_signal = np.pad(noise_signal, (0, target_length - min_length), 'constant')
        
        raw_signals.append(raw_signal)
        
        for snr in snr_levels:
            noisy_signal = add_noise(raw_signal, noise_signal, snr)
            noisy_signals_dict[snr].append(noisy_signal)
    
    return np.array(raw_signals), {snr: np.array(noisy_signals_dict[snr]) for snr in snr_levels}

# Select records and noise types for the experiment
records = ['103', '105', '111', '116', '122', '205', '213', '219', '223', '230']
noise_type = 'bw'  # Focusing on BW noise for this experiment
snr_levels = [0, 1.25, 5]
target_length = 649984

raw_signals, noisy_signals_dict = load_mit_bih_data(records, noise_type, snr_levels, target_length)

# Create datasets and dataloaders
datasets = {}
for snr in snr_levels:
    datasets[snr] = ECGDataset(raw_signals, noisy_signals_dict[snr])

dataloaders = {snr: DataLoader(dataset, batch_size=16, shuffle=True) for snr, dataset in datasets.items()}

# Define the Improved DAE class
class ImprovedDAE(nn.Module):
    def __init__(self):
        super(ImprovedDAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose1d(16, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Define the Adversarial Method Generator class (same as previous)
class GeneratorAdversarial(nn.Module):
    def __init__(self):
        super(GeneratorAdversarial, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(8, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(16, 8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(8, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Define the Proposed Method Generator class with Sy-CNN+SC architecture
class GeneratorProposed(nn.Module):
    def __init__(self):
        super(GeneratorProposed, self).__init__()
        # Symmetrical coding with skip connections (Sy-CNN+SC)
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(16, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
        # Skip connections
        self.skip_connections = nn.ModuleList([
            nn.Conv1d(16, 16, kernel_size=1),
            nn.Conv1d(32, 32, kernel_size=1),
            nn.Conv1d(64, 64, kernel_size=1),
            nn.Conv1d(128, 128, kernel_size=1),
            nn.Conv1d(256, 256, kernel_size=1),
            nn.Conv1d(512, 512, kernel_size=1)
        ])

    def forward(self, x):
        skips = []
        for i, layer in enumerate(self.encoder):
            x = layer(x)
            if i % 2 == 1:
                skips.append(self.skip_connections[i // 2](x))
        
        for i, layer in enumerate(self.decoder):
            if i % 2 == 1:
                x = layer(x + skips[-(i // 2 + 1)])
            else:
                x = layer(x)
        
        return x

# Define the Discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv1d(2, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(x.size(0), -1)

# Define the calculate_snr function
def calculate_snr(original, denoised):
    noise = original - denoised
    snr = 10 * np.log10(np.sum(original ** 2) / np.sum(noise ** 2))
    return snr

def calculate_rmse(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def train(generator, discriminator, dataloaders, num_epochs=1, lr=0.000005):
    criterion = nn.BCELoss()
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    results = {'Method': [], 'Noise_Type': [], 'SNR_Level': [], 'MIT_BIH_Record': [], 'D_loss': [], 'G_loss': [], 'SNR': [], 'RMSE': []}
    
    for snr, dataloader in dataloaders.items():
        for epoch in range(num_epochs):
            for i, (raw_signals, noisy_signals) in enumerate(dataloader):
                batch_size = raw_signals.size(0)
                
                # Ensure the signals have the same length
                min_length = min(raw_signals.shape[-1], noisy_signals.shape[-1])
                raw_signals = raw_signals[:, :min_length]
                noisy_signals = noisy_signals[:, :min_length]

                # Denoise the noisy signals
                noisy_signals = noisy_signals.unsqueeze(1)  # Add channel dimension
                raw_signals = raw_signals.unsqueeze(1)

                # Train Generator
                optimizer_G.zero_grad()
                gen_signals = generator(noisy_signals)
                
                # Update valid and fake labels to match the discriminator output size
                disc_output_size = discriminator(torch.cat((gen_signals, noisy_signals), 1)).size()
                valid = torch.ones(disc_output_size).to(gen_signals.device)
                fake = torch.zeros(disc_output_size).to(gen_signals.device)
                
                g_loss = criterion(discriminator(torch.cat((gen_signals, noisy_signals), 1)), valid)
                g_loss.backward()
                optimizer_G.step()

                # Train Discriminator
                optimizer_D.zero_grad()
                real_loss = criterion(discriminator(torch.cat((raw_signals, noisy_signals), 1)), valid)
                fake_loss = criterion(discriminator(torch.cat((gen_signals.detach(), noisy_signals), 1)), fake)
                d_loss = (real_loss + fake_loss) / 2
                d_loss.backward()
                optimizer_D.step()
                
                # Calculate SNR and RMSE
                snr_value = calculate_snr(raw_signals.squeeze().cpu().numpy(), gen_signals.squeeze().cpu().detach().numpy())
                rmse_value = calculate_rmse(raw_signals.squeeze().cpu().numpy(), gen_signals.squeeze().cpu().detach().numpy())

                # Store results
                results['Method'].append(generator.__class__.__name__)
                results['Noise_Type'].append(noise_type)
                results['SNR_Level'].append(snr)
                results['MIT_BIH_Record'].append(records[i % len(records)])
                results['D_loss'].append(d_loss.item())
                results['G_loss'].append(g_loss.item())
                results['SNR'].append(snr_value)
                results['RMSE'].append(rmse_value)
                
                print(f"[{generator.__class__.__name__} - {noise_type} SNR {snr}] [Epoch {epoch + 1}/{num_epochs}] [Batch {i + 1}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}] [SNR: {snr_value}] [RMSE: {rmse_value}]")

    return results

# Initialize models for each method
generator_dae = ImprovedDAE()
generator_adversarial = GeneratorAdversarial()
generator_proposed = GeneratorProposed()
discriminator = Discriminator()

# Train each model and collect results
results_dae = train(generator_dae, discriminator, dataloaders)
results_adversarial = train(generator_adversarial, discriminator, dataloaders)
results_proposed = train(generator_proposed, discriminator, dataloaders)

# Combine results into a DataFrame
df_results_dae = pd.DataFrame(results_dae)
df_results_adversarial = pd.DataFrame(results_adversarial)
df_results_proposed = pd.DataFrame(results_proposed)
df_results = pd.concat([df_results_dae, df_results_adversarial, df_results_proposed])

# Save the results
df_results.to_csv('denoising_results.csv', index=False)

# Display the DataFrame as a table
import ace_tools as tools; tools.display_dataframe_to_user(name="Denoising Results", dataframe=df_results)


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\malik\AppData\Roaming\Python\Python311\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.